In [1]:
import pandas as pd
data = pd.read_csv("data/file.csv", encoding='latin-1')

In [2]:
data = data[data["Unit_x"] == "tonnes"]
data = data.drop("Domain", axis= 1)
data = data.drop("Unit_y", axis= 1)
data = data.drop("Item_x", axis=1)


In [3]:
from sklearn.preprocessing import OneHotEncoder
yield_df_onehot = pd.get_dummies(data, columns=['Area',"Unit_x", "Element_x","Element_y","Item_y"], prefix = ['Area', "Unit_x","Element_x","Element_y" ,"Item_y"])


In [4]:
yield_df_onehot

,Year,avg_temp,yield_amound,pesticide_amound,Area_Albania,Area_Algeria,Area_Angola,Area_Argentina,Area_Australia,Area_Austria,...,Item_y_Rodenticides Ã¢ÂÂ Other,Item_y_Seed Treat Fung Ã¢ÂÂ Benzimidazoles,Item_y_Seed Treat Fung Ã¢ÂÂ Botanical products and biologicals,Item_y_Seed Treat Fung Ã¢ÂÂ Dithiocarbamates,Item_y_Seed Treat Fung Ã¢ÂÂ Other,"Item_y_Seed Treat Fung Ã¢ÂÂ Triazoles, diazoles",Item_y_Seed Treat Insect Ã¢ÂÂ Carbamates,Item_y_Seed Treat Insect Ã¢ÂÂ Organo-phosphates,Item_y_Seed Treat Insect Ã¢ÂÂ Other,Item_y_Seed Treat Insect Ã¢ÂÂ Pyrethroids
0,2005,26.72,4620100.00,0.00,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,1992,14.97,72807.00,12.00,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,1990,18.29,700.00,0.00,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,1997,5.19,14263885.00,97807.33,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,1994,26.03,2182130.00,2775.00,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2383935,2009,16.73,25000.00,25.22,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2383937,1997,22.90,111841.77,6853.00,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2383940,1992,16.14,24000.00,12.00,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2383943,1992,25.69,2508184.00,0.00,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [9]:
X 


,Year,avg_temp,pesticide_amound,Area_Albania,Area_Algeria,Area_Angola,Area_Argentina,Area_Australia,Area_Austria,Area_Bahamas,...,Item_y_Rodenticides Ã¢ÂÂ Other,Item_y_Seed Treat Fung Ã¢ÂÂ Benzimidazoles,Item_y_Seed Treat Fung Ã¢ÂÂ Botanical products and biologicals,Item_y_Seed Treat Fung Ã¢ÂÂ Dithiocarbamates,Item_y_Seed Treat Fung Ã¢ÂÂ Other,"Item_y_Seed Treat Fung Ã¢ÂÂ Triazoles, diazoles",Item_y_Seed Treat Insect Ã¢ÂÂ Carbamates,Item_y_Seed Treat Insect Ã¢ÂÂ Organo-phosphates,Item_y_Seed Treat Insect Ã¢ÂÂ Other,Item_y_Seed Treat Insect Ã¢ÂÂ Pyrethroids
0,2005,26.72,0.00,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,1992,14.97,12.00,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,1990,18.29,0.00,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,1997,5.19,97807.33,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,1994,26.03,2775.00,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2383935,2009,16.73,25.22,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2383937,1997,22.90,6853.00,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2383940,1992,16.14,12.00,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2383943,1992,25.69,0.00,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
# Load the data into a pandas DataFrame
import seaborn as sns

# Extract features (X) and target variable (y)
X = yield_df_onehot.drop("yield_amound", axis=1)
y = yield_df_onehot['yield_amound']
scaler = StandardScaler().fit(X_train)
Xstd=scaler.transform(X)
#Xstd=scaler.transform(X)
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(Xstd, y, test_size=0.2, random_state=42)

# Initialize the linear regression model
model = LinearRegression()

# Fit the model to the training data
model.fit(X_train, y_train)

# Predict on the test data
y_pred = model.predict(X_test)

# Evaluate the model using mean squared error
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)

Mean Squared Error: 1233385446513209.0


In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
# Build the neural network model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

# Predict on the test data
y_pred = model.predict(X_test)

# Evaluate the model using mean squared error
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)

Epoch 1/10
22253/22253 [==============================] - 27s 1ms/step - loss: 1273902466596864.0000
Epoch 2/10
22253/22253 [==============================] - 26s 1ms/step - loss: 1241446338265088.0000
Epoch 3/10
22253/22253 [==============================] - 26s 1ms/step - loss: 1240311393157120.0000
Epoch 4/10
22253/22253 [==============================] - 26s 1ms/step - loss: 1239650907717632.0000
Epoch 5/10
22253/22253 [==============================] - 26s 1ms/step - loss: 1239211210440704.0000
Epoch 6/10
22253/22253 [==============================] - 26s 1ms/step - loss: 1238946801516544.0000
Epoch 7/10
22253/22253 [==============================] - 26s 1ms/step - loss: 1238733261111296.0000
Epoch 8/10
22253/22253 [==============================] - 26s 1ms/step - loss: 1238572065619968.0000
Epoch 9/10
22253/22253 [==============================] - 25s 1ms/step - loss: 1238449390616576.0000
Epoch 10/10
5564/5564 [==============================] - 5s 849us/step
Mean Squared Error: 

In [17]:
1273902466596864.0000-1238375705083904.0000

35526761512960.0